# 패키지 불러오기

In [1]:
!openai --version

openai 1.35.7


In [43]:
import os
import yaml
import json

from openai import OpenAI
import pandas as pd
import numpy as np
import ipywidgets as widgets # interactive display
from tqdm import tqdm # progress bar
from dotenv import load_dotenv
from collections import defaultdict

# ENV, Config 파일 읽기

In [3]:
# .env 파일에서 환경 변수 로드(API 키)
load_dotenv()

True

In [4]:
# YAML 파일 열기
yaml_path = 'config.yaml' # todo: config 파일과 합치기

with open(yaml_path, 'r') as f:
    config = yaml.safe_load(f)

# gpt prompt 설정
sentiment_instructions = config['gptapi']['sentiment']['instructions']
sentiment_few_shots = config['gptapi']['sentiment']['few_shot_examples']
theme_instructions = config['gptapi']['theme']['instructions']
theme_few_shots = config['gptapi']['theme']['few_shot_examples']


# 인스턴스 생성

In [5]:
# OpenAI API 클라이언트 생성
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# 함수 정의

In [6]:
def get_response(messages):
    """
    model: 모델 종류
    messages: 사용자의 입력과 모델의 출력을 교환하는 메시지 목록
    max_tokens: 생성될 응답의 최대 길이
    temperature: 생성될 응답의 다양성(0.0 ~ 1.0) 0.0은 가장 확실한 답변을, 1.0은 가장 다양한 답변을 생성
    stream: 응답을 한 번에 반환할지 여부. False로 설정하면 한 번에 반환
    """
    try:
        response = client.chat.completions.create(
            model = "gpt-4o",
            messages = messages,
            # max_tokens = 150,
            temperature = 0.5,
            stream = False)
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# JSON 응답에서 raw_text 항목을 추출하는 함수
def extract_raw_text(content):
    return content.get('raw_text', '')

# JSON 응답에서 issue 항목을 추출하는 함수
def extract_issues(content):
    return content.get('issue', [])

# JSON 응답에서 각 issue에 대한 sentiment 값을 추출하는 함수
def extract_sentiments(content):
    return content.get('sentiment', {})

# JSON 응답에서 sentiment_all 값을 추출하는 함수
def extract_sentiment_all(content):
    return content.get('sentiment_all', '')

# 문자열을 딕셔너리로 변환하는 함수
def convert_to_dict(content):
    try:
        return json.loads(content)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return None
    
# JSON 데이터를 변환하고 추출된 데이터를 DataFrame으로 변환하는 함수
def convert_to_dataframe(content_list):
    data = []
    
    for content in content_list:
        if not content:
            continue  # content가 None이거나 비어 있을 때 건너뜀
        
        content_dict = convert_to_dict(content)
        if not content_dict:
            continue  # JSON 변환에 실패한 경우 건너뜀
        
        raw_text = extract_raw_text(content_dict)
        issues = extract_issues(content_dict)
        sentiments = extract_sentiments(content_dict)
        sentiment_all = extract_sentiment_all(content_dict)
        
        # 각 issue와 sentiment를 별도로 저장
        for issue, sentiment in sentiments.items():
            data.append({
                "raw_text": raw_text,
                "issue": issue,
                "sentiment": sentiment,
                "sentiment_all": sentiment_all
            })
    
    # 데이터가 있을 경우에만 DataFrame 생성
    if data:
        return pd.DataFrame(data)
    else:
        print("No valid data to create DataFrame.")
        return pd.DataFrame()  # 빈 DataFrame 반환
    
def map_issues_to_themes(issues_list, clustering_response):
    themes = []
    for issues in issues_list:
        issue_themes = set()  # 중복을 방지하기 위해 set 사용
        for issue in issues:
            for theme, issue_group in clustering_response.items():
                if issue in issue_group:
                    issue_themes.add(theme)
        themes.append(", ".join(issue_themes))  # 여러 테마가 있을 경우 ","로 결합
    return themes

# 후처리 함수 정의(리스트, 딕셔너리 형태를 문자열로 변환)
def clean_format(value):
    if isinstance(value, list):
        return ', '.join(value)  # 리스트의 경우 ,로 구분하여 문자열로 변환
    if isinstance(value, dict):
        return ', '.join([f"{k}: {v}" for k, v in value.items()])  # 딕셔너리의 경우 key: value 형식으로 변환
    return value  # 이미 문자열인 경우는 그대로 반환

In [7]:
def classify_issues(issues):
    # 'instructions'는 모델의 행동 방침을 설정하므로 system 메시지에 포함
    messages = [
        {"role": "system", "content": theme_instructions},
        {"role": "user", "content": f"다음 이슈들을 의미적으로 비슷한 그룹으로 나누고, 각 그룹의 공통 테마를 도출해줘: {issues}"},
        {"role": "user", "content": f"예시: {theme_few_shots}"}
    ]
    
    return get_response(messages)

# themes 리스트의 각 JSON 문자열을 dict 형태로 변환하여 병합하는 함수
def merge_themes(themes):
    merged_dict = defaultdict(list)  # 리스트를 기본값으로 갖는 defaultdict 생성

    for theme in themes:
        # JSON 문자열을 Python dict로 변환
        theme_dict = json.loads(theme)
        
        # 각 key와 value를 병합
        for key, values in theme_dict.items():
            for value in values:
                if value not in merged_dict[key]:  # 중복된 value가 없을 경우만 추가
                    merged_dict[key].append(value)
    
    return dict(merged_dict)  # defaultdict을 일반 dict로 변환하여 반환

# 'theme' 컬럼을 생성하는 함수(이슈를 테마로 매핑)
def find_theme(merged_themes, issue):
    for theme, issues in merged_themes.items():
        if issue in issues:
            return theme
    return '기타'  # 해당하는 theme이 없을 경우 '기타'로 표시

# 파일 읽기

In [44]:
result = pd.read_pickle('result/제공_네이버쇼핑리뷰_S1_감성분석.pkl')

In [45]:
result

,raw_text,issue,sentiment,sentiment_all
0,조아용 조아용 조아용 ㅎㅎ,좋은 제품,긍정,긍정
1,너무 만족하며 사용중입니다,만족,긍정,긍정
2,4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요,빠른 설치 배송,긍정,긍정
3,4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요,설치 서비스 만족,긍정,긍정
4,여자친구가 너무 만족합니다. 생일때 사줄걸 그랬어요,여자친구 만족,긍정,긍정
...,...,...,...,...
2468,일단 디자인이 너무 이쁘고 기능도 많아 왜 이제 샀나 싶은 제품 이에요~ 야외도 한...,야외 사용 기대,긍정,긍정
2469,일단 디자인이 너무 이쁘고 기능도 많아 왜 이제 샀나 싶은 제품 이에요~ 야외도 한...,대대만족,긍정,긍정
2470,침대 앞에 두니 아늑한 영화관이 생겼네요-!여기저기 옮겨 다니며 보기 편해요캠핑 갈...,아늑한 영화관 느낌,긍정,긍정
2471,침대 앞에 두니 아늑한 영화관이 생겼네요-!여기저기 옮겨 다니며 보기 편해요캠핑 갈...,이동 편리,긍정,긍정


In [48]:
result.loc[result['raw_text']==""]

,raw_text,issue,sentiment,sentiment_all
78,,빠른 배송,긍정,긍정
79,,섬세한 설치,긍정,긍정
96,,배송 시간 지정 가능,긍정,긍정
97,,튼튼한 기둥,긍정,긍정
98,,좋은 화질,긍정,긍정
...,...,...,...,...
2430,,배터리 지속시간,중립,중립
2441,,기대되는 제품,긍정,긍정
2442,,라이브쇼핑 구매,중립,긍정
2443,,빠른 배송,긍정,긍정


### Step3) theme 생성(군집분석)

In [10]:
# issue에서 중복 제거
unique_issues = result['issue'].drop_duplicates().tolist()

In [11]:
len(unique_issues)

1153

In [12]:
print(theme_instructions)

너는 제품 리뷰 키워드를 군집화하고 분류하는 전문가야. 다음 지시사항에 따라 결과를 생성해줘.
<지시사항>
1. 이슈들을 의미적으로 비슷한 그룹으로 나누고, 각 그룹의 공통 테마(theme)를 도출해줘
2. 답변은 코드블록이나 마크다운 포맷 없이, 순수한 JSON 포맷으로 제공한다. ```json 같은 형식은 사용하지 않는다.
</지시사항>



In [13]:
print(theme_few_shots)

{
  "화질 만족": ["화질 우수", "좋은 화질", "선명한 화질"],
  "사용 편의성": ["리모컨 사용 편리", "메뉴 구성 직관적"],
  "배송 서비스": ["빠른 배송", "친절한 배송"]
}



In [14]:
unique_issues

['좋은 제품',
 '만족',
 '빠른 설치 배송',
 '설치 서비스 만족',
 '여자친구 만족',
 '선물 아쉬움',
 '빠른 배송',
 '직접 배송 및 설치',
 '더 큰 화면으로 OTT 시청 좋음',
 '이동 편리',
 'OTT 시청',
 '디자인 만족',
 '품질 만족',
 '빠른 설치 안내',
 '저렴한 가격',
 '예쁜 디자인',
 '배송 만족',
 '충전 번거로움',
 '대만족',
 '친절한 설치',
 '빠른 설치',
 '좋은 가격',
 '좋은 서비스',
 '조립 서비스',
 '적극 추천',
 '무겁지만 이동 용이',
 '우수한 사운드',
 '우수한 화질',
 '식사시간 시청',
 '자기전 시청',
 '친절한 서비스',
 '좋은 TV',
 '제품 만족',
 '신속한 설치',
 '설명 만족',
 '배송 설치',
 '친절한 배송',
 '할인',
 '친절한 설치 기사',
 '편리한 터치',
 '부모님 선물 추천',
 '만족스러운 소비',
 '추천',
 '친절한 기사',
 '빠른 구매',
 'OTT/유튜브 시청 좋음',
 '리모컨',
 '터치 작동',
 '누워서 사용하기 좋음',
 'TV 미시청자 추천',
 '실용성',
 '훌륭한 디자인',
 '내구성',
 'TV없는 가정에서 유용',
 '편리한 이동',
 '만족스러운 화질',
 '만족스러운 음량',
 '강추',
 '섬세한 설치',
 '큰 화면으로 시청',
 '안전한 배송',
 '설치 만족',
 'OTT 시청 편리',
 '전용티비',
 '가치 있음',
 '적기에 상품 구입',
 '운동 시 사용',
 '넷플릭스 시청',
 '배송 시간 지정 가능',
 '튼튼한 기둥',
 '좋은 화질',
 '좋은 소리',
 '좁은 방에서 스피커 불필요',
 '아이들 교육에 유용',
 '지상파 미시청',
 '유료 어플 필요',
 '이동식 티비 만족',
 '편리한 사용',
 '다양한 기기 사용',
 '친절한 판매자',
 '친절한 설치기사',
 '뒷면 색상',
 '구매 만족',
 '거실 시청',
 '안방 시청',
 '추천 의사',


### batch100

In [15]:
# 유니크한 이슈들을 GPT API로 분류
themes = []
batch_size = 100  # 한번에 너무 많은 데이터를 보내지 않기 위해 배치 처리
for i in range(0, len(unique_issues), batch_size):
    batch_issues = unique_issues[i:i+batch_size]
    themes.append(classify_issues(', '.join(batch_issues)))

os.system('say "작업이 완료되었습니다."')
# # 배치 결과를 합쳐서 하나의 테마 리스트로 변환
# theme_list = [theme.strip() for theme in ', '.join(themes).split(',')]

0

In [16]:
themes

['{\n  "제품 품질 및 성능": ["좋은 제품", "우수한 사운드", "우수한 화질", "좋은 화질", "좋은 소리", "만족스러운 화질", "만족스러운 음량", "깨끗한 화면"],\n  "디자인 및 인테리어": ["디자인 만족", "예쁜 디자인", "훌륭한 디자인", "깔끔한 디자인", "인테리어와 조화", "뒷면 색상"],\n  "가격 및 가치": ["저렴한 가격", "좋은 가격", "할인", "가격 만족", "가치 있음"],\n  "배송 및 설치 서비스": ["빠른 설치 배송", "설치 서비스 만족", "빠른 배송", "직접 배송 및 설치", "빠른 설치 안내", "배송 만족", "신속한 설치", "배송 설치", "친절한 배송", "친절한 설치 기사", "안전한 배송", "배송 시간 지정 가능", "친절한 설치", "설치 만족", "섬세한 설치"],\n  "사용 편의성": ["이동 편리", "편리한 터치", "터치 작동", "누워서 사용하기 좋음", "편리한 이동", "편리한 사용", "무선 기능", "조용한 이동"],\n  "고객 만족 및 추천": ["만족", "여자친구 만족", "대만족", "적극 추천", "추천", "만족스러운 소비", "강추", "추천 의사", "만족스러운 구매", "구매 만족", "잘 사용 중"],\n  "서비스 및 지원": ["친절한 설치", "친절한 서비스", "친절한 기사", "친절한 판매자", "친절한 설치기사"],\n  "OTT 및 TV 시청": ["더 큰 화면으로 OTT 시청 좋음", "OTT 시청", "OTT/유튜브 시청 좋음", "넷플릭스 시청", "TV 시청 가능", "TV없는 가정에서 유용", "전용티비", "지상파 미시청", "아이들 교육에 유용"],\n  "제품 활용": ["식사시간 시청", "자기전 시청", "운동 시 사용", "거실 시청", "안방 시청", "좁은 공간에서 활용 좋음"],\n  "기타": ["선물 아쉬움", "충전 번거로움", "유료 어플 필요", "좁은 방에서 스피커 불필요", "구

In [17]:
# 각 theme의 key를 추출하여 총 길이를 계산
total_keys_length = sum(len(json.loads(theme).keys()) for theme in themes)
total_keys_length

116

In [18]:
def merge_themes(themes):
    merged_dict = defaultdict(list)  # 리스트를 기본값으로 갖는 defaultdict 생성

    for theme in themes:
        # JSON 문자열을 Python dict로 변환
        theme_dict = json.loads(theme)
        
        # 각 key와 value를 병합
        for key, values in theme_dict.items():
            for value in values:
                if value not in merged_dict[key]:  # 중복된 value가 없을 경우만 추가 todo: 판단의 논리가 필요함.
                    merged_dict[key].append(value)
    
    return dict(merged_dict)  # defaultdict을 일반 dict로 변환하여 반환

In [19]:
merged_themes = merge_themes(themes)

In [20]:
result_test = result.copy()

In [21]:
# 'theme' 컬럼을 생성
result_test['theme'] = result_test['issue'].apply(lambda x: find_theme(merged_themes, x))
result_test

,raw_text,issue,sentiment,sentiment_all,theme
0,조아용 조아용 조아용 ㅎㅎ,좋은 제품,긍정,긍정,제품 품질 및 성능
1,너무 만족하며 사용중입니다,만족,긍정,긍정,고객 만족 및 추천
2,4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요,빠른 설치 배송,긍정,긍정,배송 및 설치 서비스
3,4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요,설치 서비스 만족,긍정,긍정,배송 및 설치 서비스
4,여자친구가 너무 만족합니다. 생일때 사줄걸 그랬어요,여자친구 만족,긍정,긍정,고객 만족 및 추천
...,...,...,...,...,...
2468,일단 디자인이 너무 이쁘고 기능도 많아 왜 이제 샀나 싶은 제품 이에요~ 야외도 한...,야외 사용 기대,긍정,긍정,캠핑 및 야외 활용
2469,일단 디자인이 너무 이쁘고 기능도 많아 왜 이제 샀나 싶은 제품 이에요~ 야외도 한...,대대만족,긍정,긍정,제품 만족도
2470,침대 앞에 두니 아늑한 영화관이 생겼네요-!여기저기 옮겨 다니며 보기 편해요캠핑 갈...,아늑한 영화관 느낌,긍정,긍정,디자인 및 인테리어
2471,침대 앞에 두니 아늑한 영화관이 생겼네요-!여기저기 옮겨 다니며 보기 편해요캠핑 갈...,이동 편리,긍정,긍정,사용 편의성


In [22]:
result_test['theme'].value_counts()

theme
기타             584
배송 및 설치 서비스    310
제품 품질 및 성능     190
사용 편의성         168
고객 만족 및 추천     111
              ... 
가격 및 판매자 소통      2
브랜드 및 신뢰         2
배송 관련            2
어플 및 소프트웨어       1
내구성              1
Name: count, Length: 82, dtype: int64

In [23]:
# theme가 기타인 데이터 추출
result_test[result_test['theme'] == '기타']

,raw_text,issue,sentiment,sentiment_all,theme
5,여자친구가 너무 만족합니다. 생일때 사줄걸 그랬어요,선물 아쉬움,중립,긍정,기타
14,핸드폰으로 ott를 보다보니 항상 아쉬웠는데 고민고민하다질렀어요~^^ 주문후 다음날...,품질 만족,긍정,긍정,기타
19,충전이 번거롭긴 하지만 대만족,충전 번거로움,부정,긍정,기타
22,친절하게 설치해주시구 무었보다 빠르게 설치해주셔서 넘 좋았어요 >__< 좋은가격에 ...,빠른 설치,긍정,긍정,기타
24,친절하게 설치해주시구 무었보다 빠르게 설치해주셔서 넘 좋았어요 >__< 좋은가격에 ...,좋은 서비스,긍정,긍정,기타
...,...,...,...,...,...
2440,설치 빠르고 좋습니다.,빠른 설치,긍정,긍정,기타
2445,외이프 형님 선물로 드렸는데 넘나 만족해하심ㅎ캠핑용~~~~,선물 만족,긍정,긍정,기타
2454,너무좋아요 날시원해지면 캠핑 갈 생각에 행복,캠핑 기대,긍정,긍정,기타
2463,물건 잘 받았습니다 배송도 좋게 보내 주시고 제품도 좋고 조금 아쉬운 게 충전 충정...,제품 만족,긍정,중립,기타


In [35]:
# "raw_text" 값에 따른 고유 번호 생성 (0부터 시작).
unique_ids, _ = pd.factorize(result_test['raw_text']) # 인덱스만 사용하므로 두 번째 반환값은 무시

# "tid" 값 생성 (0001, 0002 형태로 고유 번호 할당)
result_test['tid'] = ['text' + str(i+1).zfill(4) for i in unique_ids]

# "tid" 컬럼을 "raw_text" 앞에 삽입
result_test = result_test[['tid'] + [col for col in result_test.columns if col != 'tid']]
result_test

           tid                                           raw_text       issue  \
0     text0001                                     조아용 조아용 조아용 ㅎㅎ       좋은 제품   
1     text0002                                     너무 만족하며 사용중입니다          만족   
2     text0003      4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요    빠른 설치 배송   
3     text0003      4일이내 설치배송연락받았고 원하는날 안착완료^^제품 설치 서비스 모두모두 만족이요   설치 서비스 만족   
4     text0004                       여자친구가 너무 만족합니다. 생일때 사줄걸 그랬어요     여자친구 만족   
...        ...                                                ...         ...   
2468  text0911  일단 디자인이 너무 이쁘고 기능도 많아 왜 이제 샀나 싶은 제품 이에요~ 야외도 한...    야외 사용 기대   
2469  text0911  일단 디자인이 너무 이쁘고 기능도 많아 왜 이제 샀나 싶은 제품 이에요~ 야외도 한...        대대만족   
2470  text0912  침대 앞에 두니 아늑한 영화관이 생겼네요-!여기저기 옮겨 다니며 보기 편해요캠핑 갈...  아늑한 영화관 느낌   
2471  text0912  침대 앞에 두니 아늑한 영화관이 생겼네요-!여기저기 옮겨 다니며 보기 편해요캠핑 갈...       이동 편리   
2472  text0912  침대 앞에 두니 아늑한 영화관이 생겼네요-!여기저기 옮겨 다니며 보기 편해요캠핑 갈...     캠핑에서 유용   

     sentiment sentiment_al

In [27]:
result_test['raw_text'].nunique()

912

In [42]:
result_test.to_pickle('result/제공_네이버쇼핑리뷰_S2_batch100_감성테마.pkl')

In [41]:
result_test.to_excel('result/제공_네이버쇼핑리뷰_S2_batch100_감성테마.xlsx', index = False)